## Spectral Clustering
This is a very brief and simple demo of spectral clustering for graphs. 
None of the code below is particularly optimized in any way for running time, memory etc.

(c) M. Schaub, IDSS, Massachusetts Institute of Technology

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This code is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.
http://www.gnu.org/licenses/.


In [ ]:
#Importing some libraries first 
#networks library
import networkx as nx             
#a rule to show plots inside browser
%matplotlib inline
#plotting library
import matplotlib.pyplot as plt
from IPython import display


#linear algebra etc. --- importing with * is for convenience, even though not recommeded!
import numpy as np
import scipy.linalg
import scipy.sparse as sparse

print 'Libraries imported'

In [ ]:
def create_block_model_graph(affinity_matrix, nr_nodes=100):
    # affinity matrix, partition_vec and degree_seq should all be numpy arrays
    # UNCORRECTED SBM -- affinity matrix should specify the link probability between each block.
    
    nr_groups = affinity_matrix.shape[0]
    group_size = nr_nodes / nr_groups
    rem = nr_nodes % nr_groups
    group_size_vec= np.ones(nr_groups,dtype=int)*group_size
    group_size_vec[-1] += rem
    
    # true partition vector
    partition_vec = np.hstack([np.zeros((1,rem)), np.kron(np.arange(nr_groups),np.ones((1,group_size)))]).flatten()
    
    data = np.tile(1,(nr_nodes,1)).reshape(nr_nodes)
    partition_matrix = sparse.coo_matrix((data,(np.arange(nr_nodes),partition_vec))).toarray()
    
    P = partition_matrix.dot(affinity_matrix).dot(partition_matrix.T)
    
    G=nx.Graph()
    # make sure that all nodes are added even though the graph might be disconnected
    G.add_nodes_from(np.arange(nr_nodes))
            

    #cycle through nodes and generate edges
    for i in range(nr_groups):
        for j in range(i,nr_groups):
            p = affinity_matrix[i,j]
            # sample edges in each block at random as a vector, reshape it to a matrix
            # and then construct a network from it
            edge_vec = np.random.rand((group_size_vec[i]*group_size_vec[j])) < p
            edges = edge_vec.reshape((group_size_vec[i],group_size_vec[j]))
            if i == j:
                edges = np.triu(edges)
            edges = edges.nonzero()
            G.add_edges_from(zip(i*group_size+edges[0],j*group_size+edges[1]))

        #remove self loops
        G.remove_edges_from(G.selfloop_edges())
    return G, P


### Example: stochastic blockmodel
A stochastic blockmodel (SBM) is a generative model for a network. 
The model is specified as follows.
Each node $i$ is endowed with a (latent) group label $g_i = \{1,\ldots,k\}$.
The probability for two nodes $i, j$ to connect, conditional on the group labels, is given by:

$$\mathbb P(A_{ij} = 1 | g_i, g_j) = \omega_{g_i g_j}$$

where we have defined the affinity matrix $(\omega_{ij})$, where each entry $\omega_{ij}\in [0,1]$. 
Further, all edges are (conditionally) idependently distributed.

In our example below we will focus on the case where the diagonal entries of $\omega$ are larger than the off-diagonal terms ('clustering').

In [ ]:
# Let's create a graph with blockmodel structure
affinity = np.array([[0.8,.05, 0],[.05,0.8, 0.1],[0, 0.1,0.7]])
print affinity

G, P = create_block_model_graph(affinity, nr_nodes=500)
position=nx.spring_layout(G,scale=5)
# drawing may be slow..
nx.draw_networkx(G, pos=position, cmap=plt.get_cmap('jet')) 

Let us define the $\{0,1\}$ partition indicator matrices $H_{ij} = 1$ if node $i$ is in group j, and zero otherwise.
The expected adjacency matrix for our graph (conditioned on the group labels) is now:

$$ \mathcal A =\mathbb E [A|\{g_i\}] = H \omega H^\top$$

In [ ]:
#The expected adjacency matrix looks like this..
plt.figure()
plt.imshow(P);
plt.colorbar();

### Recovering the partitions from the expected adjacency matrix
Let's see how we could recover the partition via a spectral algorithm, if we had access to $\mathcal A$.

In [ ]:
D = np.diag(P.sum(axis=1))
LP = D - P

# first plot --- the spectrum of the expected adjacency matrix -- there are only k nonzero eigenvalues
ev, evecs = scipy.linalg.eigh(P)
plt.figure()
plt.plot(ev);

# the corresponding Laplacian spectrum --- now we have to look for the smallest eigenvalues..
ev, evecs = scipy.linalg.eigh(LP)
plt.figure()
plt.plot(ev);

# Let us check the corresponding eigenvectors --- there should be k (k=3) eigenvectors which are constant on each group...
plt.figure()
plt.plot(evecs[:,:4]);

As we can see, there are indeed 3 piecewise constant eigenvectors -- what now?
Let's denote these three vectors by $V_i, i\in 1,\ldots, k$ and form the matrix $V = [V_1,\ldots,V_k]$.
Now we assign each node $i$ in the network a coordinate corresponding to the $i$th row of this matrix $V$.
(This is a spectral embedding).

We can plot the nodes in this lower-dimensional space.

In [ ]:
# Let's take the 3 eigenvectors and use the entries as coordinates
# As there the eigenvectors were constant on the groups, there are only 3 distinct positions a node can occupy
X = evecs[:,:3]
plt.figure();
plt.plot(X[:,1],X[:,2],'rx');

### Recovering the partitions from the observed network
Of course, we normally not have access to the generative model and the expected network --- what can we do??
Let us assume here that the network came from a SBM. Subject to some technical conditions, one can then show that the observed network will have spectral properties that are close to the expected network. So instead of using the eigenvectors of the expected adjacency matrix, we will make use of the observed matrix.

In [ ]:
#Let's now do the same thing with the observed (sampled) network
A = nx.to_scipy_sparse_matrix(G)
D = np.diag(A.sum(axis=0))
L = D-A

ev, evecs = scipy.linalg.eigh(L)
plt.figure();
plt.plot(ev);

plt.figure();
plt.plot(evecs[:,:3]);

# Let's take the 3 eigenvectors and use the entries as coordinates
# Here we use only the 2nd and 3 dimension again
X = evecs[:,:3]
plt.figure();
plt.plot(X[:,1],X[:,2],'rx');

In [ ]:
# Let's do all of this on the observed network, with a 'full' algorithm -- as proposed by Qin and Rohe.
# Note that this algorithm uses a few additional tricks, such as regularization (works better for sparse networks),
# and is based on the degree corrected blockmodel.

from __future__ import division
import numpy as np
import scipy.sparse
import scipy.sparse.linalg as linalg
import networkx as nx
from sklearn.cluster import KMeans
from sklearn import preprocessing

##########################################
# REGULARIZED SPECTRAL CLUSTERING (ROHE)
##########################################

def regularized_laplacian_spectral_clustering(A, num_groups=2, tau=-1):
    """
    Performs regularized spectral clustering based on Qin-Rohe 2013 using a normalized and
    regularized adjacency matrix (called Laplacian by Rohe et al)
    """

    A = test_sparse_and_transform(A)

    # check if tau regularisation parameter is specified otherwise go for mean degree...
    if tau==-1:
        # set tau to average degree
        tau = A.sum()/A.shape[0]

    d = np.array(A.sum(axis=1)).flatten().astype(float)
    Dtau_sqrt_inv = scipy.sparse.diags(np.power(d + tau,-.5),0)
    L = Dtau_sqrt_inv.dot(A).dot(Dtau_sqrt_inv)


    # compute eigenvalues and eigenvectors (sorted according to magnitude first)
    ev, evecs = scipy.sparse.linalg.eigsh(L,num_groups,which='LM')

    X = preprocessing.normalize(evecs, axis=1, norm='l2')

    clust = KMeans(n_clusters = num_groups)
    clust.fit(X)
    partition_vector = clust.labels_


    return partition_vector, evecs

def test_sparse_and_transform(A):
    """ Check if matrix is sparse and if not, return it as sparse matrix"""
    if not scipy.sparse.issparse(A):
        print "Input matrix not in sparse format, transforming to sparse matrix"
        A = scipy.sparse.csr_matrix(A)
    return A


partition,number_partition= regularized_laplacian_spectral_clustering(A, num_groups=3, tau=-1)
print "partition=", partition
nx.draw_networkx(G, pos=position, cmap=plt.get_cmap('prism'),node_color=partition,vmin=partition.min(),vmax=partition.max()) 


### Extra Material to play with 

Some further spectral clustering code to play with / adapt (make sure to load all necessary libraries first)..
For instance one can use the Bethe Hessian to estimate the number of groups present in the network.

In [ ]:
######################################
# BETHE HESSIAN CLUSTERING
######################################

def build_BetheHessian(A, r):
    """
    Construct Standard Bethe Hessian as discussed, e.g., in Saade et al
    B = (r^2-1)*I-r*A+D
    """
    A = test_sparse_and_transform(A)

    d = A.sum(axis=1).getA().flatten().astype(float)
    B = scipy.sparse.eye(A.shape[0]).dot(r**2 -1) -r*A +  scipy.sparse.diags(d,0)
    return B

def cluster_with_BetheHessian(A, num_groups=-1):
    """
    Perform one round of spectral clustering using the Bethe Hessian
    Input: adjacency matrix A, number of groups (num_groups; set to -1, if group number
    should be inferred automatically)
    """

    # set r to square root of average degree
    r = A.sum()/A.shape[0]
    r = np.sqrt(r)

    # construct both the positive and the negative variant of the BH
    if not all(A.sum(axis=1)):
        print "GRAPH CONTAINS NODES WITH DEGREE ZERO"

    BH_pos = build_BetheHessian(A,r)
    BH_neg = build_BetheHessian(A,-r)


    if num_groups ==-1:
        relevant_ev, _ = find_negative_eigenvectors(BH_pos)
        X = relevant_ev

        relevant_ev, _ = find_negative_eigenvectors(BH_neg)
        X = np.hstack([X, relevant_ev])
        num_groups = X.shape[1]

        if num_groups == 0:
            print "no indication for grouping -- return all in one partition"
            partition_vector = np.zeros(A.shape[0],dtype='int')
            return partition_vector

    else:
        # note that we combine the eigenvectors of pos/negative BH and do not use
        # information about positive / negative assortativity here
        # find eigenvectors corresponding to the algebraically smallest (most neg.) eigenvalues
        ev_pos, evecs_pos = scipy.sparse.linalg.eigsh(BH_pos,num_groups,which='SA')
        ev_neg, evecs_neg = scipy.sparse.linalg.eigsh(BH_neg,num_groups,which='SA')
        ev_all = np.hstack([ev_pos, ev_neg])
        index = np.argsort(ev_all)
        X = np.hstack([evecs_pos,evecs_neg])
        X = X[:,index]


    clust = KMeans(n_clusters = num_groups)
    clust.fit(X)
    partition_vector = clust.labels_


    return partition_vector

def find_negative_eigenvectors(M):
    """
    Given a matrix M, find all the eigenvectors associated to negative eigenvalues
    and return the tuple (evecs, evalues)
    """
    Kmax = M.shape[0]-1
    K = min(10,Kmax)
    ev, evecs = scipy.sparse.linalg.eigsh(M,K,which='SA')
    relevant_ev = np.nonzero(ev <0)[0]
    while (relevant_ev.size  == K):
        K = min(2*K, Kmax)
        ev, evecs = scipy.sparse.linalg.eigsh(M,K,which='SA')
        relevant_ev = np.nonzero(ev<0)[0]

    return evecs[:,relevant_ev], ev[relevant_ev]



__Some useful References__

Below I list merely some references to the algorithms implemented above, and a general tutorial article on spectral clustering. There is a lot more out there to explore..

A good overview article is the following

* Von Luxburg, Ulrike. "A tutorial on spectral clustering." Statistics and computing 17.4 (2007): 395-416.

The above implemented spectral algorithms are described here below

* Rohe, Karl, Sourav Chatterjee, and Bin Yu. "Spectral clustering and the high-dimensional stochastic blockmodel." The Annals of Statistics (2011): 1878-1915.

* Qin, Tai, and Karl Rohe. "Regularized spectral clustering under the degree-corrected stochastic blockmodel." Advances in Neural Information Processing Systems. 2013.

* Saade, Alaa, Florent Krzakala, and Lenka Zdeborová. "Spectral clustering of graphs with the bethe hessian." Advances in Neural Information Processing Systems. 2014.